Построить обобщенную линейную модель (GLM) для прогнозирования наступления страховых случаев на рассмотренных в ноутбуке данных. Придумать и использовать дополнительные факторы при построении модели (например, пересечения признаков или функции от них и т.д.). Оценить результаты построенной модели при помощи различных метрик (можно использовать и другие метрики помимо представленных в ноутбуке), проанализировать вероятные проблемы. 

Реализовать API для модели машинного обучения, код представить в виде пулл реквеста в репозиторий курса.

Датасет: freMPL (French Motor Personal Line datasets)

Источник данных: http://cas.uqam.ca/

Продукт: КАСКО

Набор из 10 датасетов частного французского автостраховщика. Каждый датасет содержит характеристики риска, суммы величин страховых требований и истории страховых исков по около 30 000 страховых полисов за 2004 год.
Наборы данных 1-4 не содержат информации о количестве страховых требований, а 5-10 не содержат информации по характеристикам транспортного средства. Тем не менее, наборы 5-9 имеют одинаковые пропущенные факторы, они объединены. Удалены пустые столбцы, дубликаты

## Предобработка данных

In [154]:
import numpy as np
import pandas as pd
import xgboost as xgb
from matplotlib import pyplot as plt

In [155]:
# Загрузим набор данных

df = pd.read_csv('freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           115155 non-null  float64
 1   LicAge             115155 non-null  int64  
 2   RecordBeg          115155 non-null  object 
 3   RecordEnd          59455 non-null   object 
 4   Gender             115155 non-null  object 
 5   MariStat           115155 non-null  object 
 6   SocioCateg         115155 non-null  object 
 7   VehUsage           115155 non-null  object 
 8   DrivAge            115155 non-null  int64  
 9   HasKmLimit         115155 non-null  int64  
 10  BonusMalus         115155 non-null  int64  
 11  ClaimAmount        115155 non-null  float64
 12  ClaimInd           115155 non-null  int64  
 13  ClaimNbResp        115155 non-null  float64
 14  ClaimNbNonResp     115155 non-null  float64
 15  ClaimNbParking     115155 non-null  float64
 16  Cl

В предыдущем уроке мы заметили отрицательную величину убытка для некоторых наблюдений. Заметим, что для всех таких полисов переменная "ClaimInd" принимает только значение 0. Поэтому заменим все соответствующие значения "ClaimAmount" нулями.

In [156]:
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())
NegClaimAmount.head()

Unique values of ClaimInd: [0]


,ClaimAmount,ClaimInd
82,-74.206042,0
175,-1222.585196,0
177,-316.288822,0
363,-666.758610,0
375,-1201.600604,0


In [157]:
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

Перекодируем переменные типа `object` с помощью числовых значений

In [158]:
def SeriesFactorizer(series):
    series, unique = pd.factorize(series)
    reference = {x: i for x, i in enumerate(unique)}
    print(reference)
    return series, reference

In [159]:
df.Gender, GenderRef = SeriesFactorizer(df.Gender)

{0: 'Male', 1: 'Female'}


In [160]:
df.MariStat, MariStatRef = SeriesFactorizer(df.MariStat)

{0: 'Other', 1: 'Alone'}


Для переменных, содержащих более 2 значений, различия между которыми не могут упорядочены, используем фиктивные переменные (one-hot encoding).


In [161]:
list(df.VehUsage.unique())

['Professional', 'Private+trip to office', 'Private', 'Professional run']

In [162]:
VU_dummies = pd.get_dummies(df.VehUsage, prefix='VehUsg', drop_first=False)
VU_dummies.head()

,VehUsg_Private,VehUsg_Private+trip to office,VehUsg_Professional,VehUsg_Professional run
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0


Фактор "SocioCateg" содержит информацию о социальной категории в виде кодов классификации CSP. Агрегируем имеющиеся коды до 1 знака, а затем закодируем их с помощью one-hot encoding.

[Wiki](https://fr.wikipedia.org/wiki/Professions_et_cat%C3%A9gories_socioprofessionnelles_en_France#Cr%C3%A9ation_de_la_nomenclature_des_PCS)

[Более подробный классификатор](https://www.ast74.fr/upload/administratif/liste-des-codes-csp-copie.pdf)

In [163]:
df['SocioCateg'].unique()

array(['CSP50', 'CSP55', 'CSP60', 'CSP48', 'CSP6', 'CSP66', 'CSP1',
       'CSP46', 'CSP21', 'CSP47', 'CSP42', 'CSP37', 'CSP22', 'CSP3',
       'CSP49', 'CSP20', 'CSP2', 'CSP40', 'CSP7', 'CSP26', 'CSP65',
       'CSP41', 'CSP17', 'CSP57', 'CSP56', 'CSP38', 'CSP51', 'CSP59',
       'CSP30', 'CSP44', 'CSP61', 'CSP63', 'CSP45', 'CSP16', 'CSP43',
       'CSP39', 'CSP5', 'CSP32', 'CSP35', 'CSP73', 'CSP62', 'CSP52',
       'CSP27', 'CSP24', 'CSP19', 'CSP70'], dtype=object)

In [164]:
df['SocioCateg'] = df.SocioCateg.str.slice(0,4)

In [165]:
pd.DataFrame(df.SocioCateg.value_counts().sort_values()).rename({'SocioCateg': 'Frequency'}, axis=1)

,Frequency
CSP7,14
CSP3,1210
CSP1,2740
CSP2,3254
CSP4,7648
CSP6,24833
CSP5,75456


In [166]:
df = pd.get_dummies(df, columns=['VehUsage','SocioCateg'])

Теперь, когда большинство переменных типа `object` обработаны, исключим их из набора данных за ненадобностью.

In [167]:
df = df.select_dtypes(exclude=['object'])

Для моделирования частоты убытков сгенерируем показатель как сумму индикатора того, что убыток произошел ("ClaimInd") и количества заявленных убытков по различным видам ущерба за 4 предшествующих года ("ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen").

В случаях, если соответствующая величина убытка равняется нулю, сгенерированную частоту также обнулим.

In [168]:
df['ClaimsCount'] = df.ClaimInd + df.ClaimNbResp + df.ClaimNbNonResp + df.ClaimNbParking + df.ClaimNbFireTheft + df.ClaimNbWindscreen
df.loc[df.ClaimAmount == 0, 'ClaimsCount'] = 0
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis=1, inplace=True)

In [169]:
df.drop(["Exposure", "OutUseNb"], axis=1, inplace=True)
 

In [170]:
#df['HasKmLimit'].describe()
#df.drop(["HasKmLimit"], axis=1, inplace=True)

In [171]:
df.drop([], axis=1, inplace=True)

In [172]:
pd.DataFrame(df.ClaimsCount.value_counts()).rename({'ClaimsCount': 'Policies'}, axis=1)

,Policies
0.0,104286
2.0,3529
1.0,3339
3.0,2310
4.0,1101
5.0,428
6.0,127
7.0,26
8.0,6
9.0,2


In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 21 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   LicAge                           115155 non-null  int64  
 1   Gender                           115155 non-null  int64  
 2   MariStat                         115155 non-null  int64  
 3   DrivAge                          115155 non-null  int64  
 4   HasKmLimit                       115155 non-null  int64  
 5   BonusMalus                       115155 non-null  int64  
 6   ClaimAmount                      115155 non-null  float64
 7   ClaimInd                         115155 non-null  int64  
 8   RiskArea                         115155 non-null  float64
 9   VehUsage_Private                 115155 non-null  uint8  
 10  VehUsage_Private+trip to office  115155 non-null  uint8  
 11  VehUsage_Professional            115155 non-null  uint8  
 12  Ve

In [190]:
df['HasKmLimit'].describe()

count    115155.000000
mean          0.109939
std           0.312815
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: HasKmLimit, dtype: float64

Для моделирования среднего убытка можем рассчитать его как отношение величины убытков к их частоте.

In [175]:
dfAC = df[df.ClaimsCount > 0].copy()
dfAC['AvgClaim'] = dfAC.ClaimAmount/dfAC.ClaimsCount

## Разделение набора данных на обучающую, валидационную и тестовую выборки

In [176]:
from sklearn.model_selection import train_test_split

In [177]:
# Разбиение датасета для частоты на train/val/test

x_train_c, x_test_c, y_train_c, y_test_c = train_test_split(df.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount'], axis=1), df.ClaimsCount, test_size=0.3, random_state=1)
x_valid_c, x_test_c, y_valid_c, y_test_c = train_test_split(x_test_c, y_test_c, test_size=0.5, random_state=1)

In [178]:
# Разбиение датасета для среднего убытка на train/val/test 

x_train_ac, x_test_ac, y_train_ac, y_test_ac = train_test_split(dfAC.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount', 'AvgClaim'], axis=1), dfAC.AvgClaim, test_size=0.3, random_state=1)
x_valid_ac, x_test_ac, y_valid_ac, y_test_ac = train_test_split(x_test_ac, y_test_ac, test_size=0.5, random_state=1)

In [179]:
x_train_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80608 entries, 54963 to 98539
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   LicAge                           80608 non-null  int64  
 1   Gender                           80608 non-null  int64  
 2   MariStat                         80608 non-null  int64  
 3   DrivAge                          80608 non-null  int64  
 4   HasKmLimit                       80608 non-null  int64  
 5   BonusMalus                       80608 non-null  int64  
 6   RiskArea                         80608 non-null  float64
 7   VehUsage_Private                 80608 non-null  uint8  
 8   VehUsage_Private+trip to office  80608 non-null  uint8  
 9   VehUsage_Professional            80608 non-null  uint8  
 10  VehUsage_Professional run        80608 non-null  uint8  
 11  SocioCateg_CSP1                  80608 non-null  uint8  
 12  SocioCateg_CSP

### Построение GLM для частоты страховых случаев

In [180]:
import lightgbm as lgb
lgb_train = lgb.Dataset(x_train_c, label=y_train_c)
lgb_test = lgb.Dataset(x_test_c, label=y_test_c, reference=lgb_train)
lgb_valid = lgb.Dataset(x_valid_c, label=y_valid_c, reference=lgb_train)

In [181]:
#Регрессия Пуассона обычно используется в случаях, когда зависимая переменная представляет 
#собой счетные значения и ошибки предполагаются распределенными в соответствии 
#с распределением Пуассона. Зависимая переменная должна быть неотрицательной. 

params = {'objective':'poisson',
#          'boosting_type': 'goss',
#          'top_rate': 0.3,
#          'other_rate': 0.3,
         'num_leaves':60, 
         'learning_rate': 0.01,
         'feature_fraction': 0.5,
         'bagging_fraction': 0.9,
         'bagging_freq': 1,
         'bagging_seed': 1,
         'poisson_max_delta_step': 0.8,
         'min_data': 5,
         'metric': ['rmse'],
#          'min_gain_to_split': 100,
         'num_threads': 4,
         'max_bin': 63
         }

In [182]:
lgb_model_freq = lgb.train(params, lgb_train, 10000, valid_sets=[lgb_test], early_stopping_rounds=250, verbose_eval=50)

Training until validation scores don't improve for 250 rounds.
[50]	valid_0's rmse: 0.758169
[100]	valid_0's rmse: 0.757466
[150]	valid_0's rmse: 0.756924
[200]	valid_0's rmse: 0.75654
[250]	valid_0's rmse: 0.756239
[300]	valid_0's rmse: 0.756034
[350]	valid_0's rmse: 0.755845
[400]	valid_0's rmse: 0.75572
[450]	valid_0's rmse: 0.755646
[500]	valid_0's rmse: 0.755599
[550]	valid_0's rmse: 0.755594
[600]	valid_0's rmse: 0.755586
[650]	valid_0's rmse: 0.755586
[700]	valid_0's rmse: 0.755591
[750]	valid_0's rmse: 0.755628
[800]	valid_0's rmse: 0.755694
[850]	valid_0's rmse: 0.755725
[900]	valid_0's rmse: 0.755771
Early stopping, best iteration is:
[678]	valid_0's rmse: 0.755567


In [183]:
probs = lgb_model_freq.predict(x_test_c, num_iteration=-1)

In [191]:
lgb_model_freq.save_model('lgb_model_freq.model')

In [ ]:
#lgb_model_freq2 = lgb.Booster(model_file='lgb_model_freq.model')

### Построение GLM для среднего убытка

In [185]:
lgb_train_ac = lgb.Dataset(x_train_ac, label=y_train_ac)
lgb_test_ac = lgb.Dataset(x_test_ac, label=y_test_ac, reference=lgb_train)
lgb_valid_ac = lgb.Dataset(x_valid_ac, label=y_valid_ac, reference=lgb_train)

In [186]:
#GLM с гамма-распределением используется для моделирования положительной непрерывной 
#зависимой переменной, когда ее условная дисперсия увеличивается вместе со 
#средним значением, но коэффициент вариации зависимой переменной предполагается постоянным.

params_ac = {'objective':'gamma',
             'regression application': 'gamma',
             'metric': ['rmse'],
             'num_threads': 2,
             'boosting_type': 'goss',
             'num_leaves': 10, 
             'learning_rate': 0.004,
             'feature_fraction': 0.5,
             'bagging_fraction': 0.9,
             'bagging_seed': 1,         
             'min_data': 5,
         'max_bin': 63
         }

In [187]:
lgb_model_avgclaim = lgb.train(params_ac, lgb_train_ac, 10000, valid_sets=[lgb_test_ac], early_stopping_rounds=500, verbose_eval=50)

Training until validation scores don't improve for 500 rounds.
[50]	valid_0's rmse: 3104.74
[100]	valid_0's rmse: 3103.73
[150]	valid_0's rmse: 3103.23
[200]	valid_0's rmse: 3102.94
[250]	valid_0's rmse: 3102.61
[300]	valid_0's rmse: 3102.28
[350]	valid_0's rmse: 3102.23
[400]	valid_0's rmse: 3102.04
[450]	valid_0's rmse: 3101.95
[500]	valid_0's rmse: 3101.88
[550]	valid_0's rmse: 3101.77
[600]	valid_0's rmse: 3101.93
[650]	valid_0's rmse: 3101.83
[700]	valid_0's rmse: 3101.74
[750]	valid_0's rmse: 3101.83
[800]	valid_0's rmse: 3101.75
[850]	valid_0's rmse: 3101.61
[900]	valid_0's rmse: 3101.51
[950]	valid_0's rmse: 3101.35
[1000]	valid_0's rmse: 3101.79
[1050]	valid_0's rmse: 3101.77
[1100]	valid_0's rmse: 3101.93
[1150]	valid_0's rmse: 3101.9
[1200]	valid_0's rmse: 3102.08
[1250]	valid_0's rmse: 3102.23
[1300]	valid_0's rmse: 3102.32
[1350]	valid_0's rmse: 3102.36
[1400]	valid_0's rmse: 3102.44
Early stopping, best iteration is:
[940]	valid_0's rmse: 3101.29


In [188]:
lgb_model_avgclaim.save_model('lgb_model_avgclaim.model')